In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install fiona
!pip install geopandas
!pip install folium
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install contextily
!pip install mapclassify
!pip install chart-studio
!sudo apt install libspatialindex-dev
!pip install rtree

     |████████████████████████████████| 16.7 MB 195 kB/s 
     |████████████████████████████████| 1.0 MB 7.6 MB/s 
     |████████████████████████████████| 6.3 MB 58.3 MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7097 sha256=061a251be2403ef79c5e395078bf2b32400f7252221f380f3ef43fda026bfc99
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText
     |████████████████████████████████| 28.8 MB 1.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
     |████████████████████████████████| 19.3 MB 1.1 MB/s 
     |████████████████████████████████| 64 kB 1.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=06de29276d8c35da12cd7c7bdbc8da28e7b99d68a3c69cfcd8a9dd56632fbc26
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d

In [ ]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import folium
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import chart_studio.plotly as py
import chart_studio.tools as tls
import os
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx
import pickle

# A magic functin that renders the figure in a notebook 
%matplotlib inline 

from matplotlib import pyplot as plt
from shapely.geometry import Point

from google.colab import files

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
!pip install pygeos
!sudo apt install libspatialindex-dev
!pip install rtree

     |████████████████████████████████| 2.1 MB 7.8 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
input_data = {'city':['Ann Arbor', 'Boulder', 'Chicago', 'DC', 'NYC', 'SF','Madison','Tulsa','Seattle'],
        'data_type':['Training','Training','Training','Training','Training','Training','Testing','Testing','Testing']}
input_df = pd.DataFrame(input_data)
input_df

,city,data_type
0,Ann Arbor,Training
1,Boulder,Training
2,Chicago,Training
3,DC,Training
4,NYC,Training
5,SF,Training
6,Madison,Testing
7,Tulsa,Testing
8,Seattle,Testing


In [ ]:
#n = 0

#for n in range(input_df.shape[0]):
#  city = input_df['city'][n]
#  data_type = input_df['data_type'][n]
#  df_location = "/content/gdrive/MyDrive/ML Final/Project Files/Urban-Mobility-Index/Data/"+ data_type +"/"+ city +"/Bus Stops/Density.pkl"
#  df = pd.read_pickle(df_location)
#  shpname = "/content/gdrive/MyDrive/ML Final/Project Files/Urban-Mobility-Index/Data/"+ data_type +"/"+ city +"/Bus Stops/Density.shp"
#  df.to_file(driver = 'ESRI Shapefile', filename = shpname)
#  print(city)
#  n = n + 1

Ann Arbor
Boulder
Chicago
DC
NYC
SF
Madison
Tulsa
Seattle


In [ ]:
def Intersections(city,data_type):
  shpname = "/content/gdrive/MyDrive/ML Final/Project Files/Urban-Mobility-Index/Data/"+ data_type +"/"+ city +"/Bus Stops/Density.shp"
  gdf = gpd.read_file(shpname, driver = "shapefile")
  gdf = gdf[['Name','Walk Score','Population','area','geometry','perSqKm','perThouCap']]
  nodename = "/content/gdrive/MyDrive/ML Final/Michelle_Intersections/Node_tables/"+ city + ".csv"
  nodes = pd.read_csv(nodename)
  geometry = [Point(xy) for xy in zip(nodes['lon'],nodes['lat'])]
  gdf_nodes= gpd.GeoDataFrame(nodes, crs="epsg:4326", geometry=geometry)
  gdf_nodes = gdf_nodes.to_crs('epsg:2163')
  gdf_join = gdf.sjoin(gdf_nodes, how="left")
  gdf_count = gdf_join.groupby(['Name'], as_index=False,)['osmid'].count()
  gdf = pd.merge(gdf_count, gdf, on='Name', how='outer')
  gdf = gdf.rename(columns={"osmid": "Count", "perSqKm":"Bus_perSqKm", "perThouCap": "Bus_perThouCap"})
  gdf["Inter_perSqKm"] = gdf["Count"]/gdf["area"] #per square kilometer
  gdf["Inter_perThouCap"] = gdf["Count"]*1000/gdf["Population"] #per thousand capita
  gdf = gdf.drop('Count', 1)

  #Export file
  pkl_out = "/content/gdrive/MyDrive/ML Final/Project Files/Urban-Mobility-Index/Data/"+ data_type +"/"+ city +"/Intersections/Density.pkl"
  gdf.to_pickle(pkl_out)
  print("Successfully Exported Intersection Density of " + city + ' to ' + data_type + '.')

In [ ]:
n = 0

for n in range(input_df.shape[0]):
  city = input_df['city'][n]
  data_type = input_df['data_type'][n]
  Intersections(city,data_type)
  n = n + 1

Successfully Exported Intersection Density of Ann Arbor to Training.
Successfully Exported Intersection Density of Boulder to Training.
Successfully Exported Intersection Density of Chicago to Training.
Successfully Exported Intersection Density of DC to Training.
Successfully Exported Intersection Density of NYC to Training.
Successfully Exported Intersection Density of SF to Training.
Successfully Exported Intersection Density of Madison to Testing.
Successfully Exported Intersection Density of Tulsa to Testing.
Successfully Exported Intersection Density of Seattle to Testing.
